In [3]:
import os
from runner import RunnerData, SlurmRunner
from runner.utils import submit
from ase import Atoms, db
from ase.io import read
import numpy as np
import copy
from matplotlib import pyplot as plt
import time
from ipywidgets import interact

In [3]:
forms = []
for i in os.listdir('data'):
    if i.endswith('LOCPOT'):
        forms.append('.'.join(i.split('.')[:-1]))
forms.sort()

In [4]:
for i, ff in enumerate(forms, start=1):
    with open(f'data/{ff}.LOCPOT', 'r') as fio:
        print(i, ff, float([fio.readline() for _ in range(5)][-1].split()[-1]))

1 Adam_1x1_O-C-Ca_dpp 45.0
2 Adam_1x1_dpp 45.0
3 Adam_2x1_O-C-Ca1.1_dpp 45.0
4 Adam_2x1_O-C-Ca1.2_dpp 45.0
5 Adam_2x1_O-C-Ca1_dpp 45.0
6 Adam_2x1_O-C-Ca2_dpp 45.0
7 Adam_2x1_O-C-Ca3_dpp 45.0
8 Adam_2x1_O-C-Ca4_dpp 45.0
9 Adam_2x1_dpp 45.0
10 Adam_CO_Ca1_Tilt1_pg_dpp 45.0
11 Adam_CO_Ca1_Tilt2_pg_dpp 45.0
12 Adam_CO_Ca1_pg_dpp 45.0
13 Adam_CO_Ca2_pg_dpp 45.0
14 Adam_CO_Ca3_pg_dpp 45.0
15 Adam_CO_Ca4_pg_dpp 45.0
16 Adam_conf10_dpp 45.0
17 Adam_conf11_dpp 45.0
18 Adam_conf12_dpp 25.0
19 Adam_conf13_dpp 25.0
20 Adam_conf1_dpp 25.0
21 Adam_conf2_dpp 25.0
22 Adam_conf3_dpp 25.0
23 Adam_conf4_dpp 25.0
24 Adam_conf5_dpp 45.0
25 Adam_conf6_dpp 45.0
26 Adam_conf7_dpp 45.0
27 Adam_conf8_dpp 45.0
28 Adam_conf9_dpp 45.0
29 Adam_high_tolerance_dpp 45.0
30 Adam_hse_dpp 25.0


In [4]:
fdb = db.connect('database.db')
if 'database.db' not in os.listdir():
    fdb.metadata = {'default_columns': ['id', 'user', 'formula', 'status']}

# Runner

In [5]:
# runner
# runner
pre_runner_data = RunnerData()
pre_runner_data.append_tasks('shell', 'module load anaconda')
pre_runner_data.append_tasks('shell', 'module load gcc')
runner = SlurmRunner('PPM',
                     pre_runner_data=pre_runner_data,
                     cycle_time=900,
                     max_jobs=50)
runner.to_database()

# RunnerData

In [6]:
params = {'ChargeCuUp': -0.0669933,
          'ChargeCuDown': -0.0627402,
          'Ccharge': 0.212718,
          'Ocharge': -0.11767,
          'Cklat': 0.0925,
          'Oklat': 0.0908,
          'CuUpshift': 2.2422001068,
          'rC0': 1.85,
          'rO0': 1.15,
          'rOx': 0,
          'rOy': 0,
          'sigma': 0.71,
          'Ckrad': 20,
          'Okrad': 20,
          'Amp': 7,
          'gridz': 45
         }

In [7]:
runner_data = RunnerData('get_ppm_data')
runner_data.scheduler_options = {'-n': 1,
                                 '--time': '0-00:30:00',
                                 '--mem-per-cpu': 12000}
# runner_data.parents = [1]
runner_data.add_file('get_data.py')
runner_data.add_file('gen_params.py')
runner_data.add_file('run.sh', 'run_PPM.sh')
runner_data.append_tasks('python', 'gen_params.py', copy.deepcopy(params))
runner_data.append_tasks('shell', 'chmod +x prepare.sh')
runner_data.append_tasks('shell', 'chmod +x run_PPM.sh')
runner_data.append_tasks('shell', './prepare.sh')
runner_data.append_tasks('shell', './run_PPM.sh')
runner_data.append_tasks('python', 'get_data.py')
runner_data.append_tasks('shell', 'if [ -d PPM-complex_tip ]; then rm -rf PPM-complex_tip; fi')
runner_data.append_tasks('shell', 'if [ -d PPM-OpenCL ]; then rm -rf PPM-OpenCL; fi')
runner_data.keep_run = True

In [8]:
for ff in forms:
    if fdb.count(label=ff) == 0:
        with open(f'data/{ff}.LOCPOT', 'r') as fio:
            z = float([fio.readline() for _ in range(5)][-1].split()[-1])
        if z < 30:
            runner_data.tasks[0][2]['Amp'] = 3.2
            runner_data.data['tasks'][0][2]['gridz'] = 25.0
        else:
            runner_data.tasks[0][2]['Amp'] = 7
            runner_data.data['tasks'][0][2]['gridz'] = 45.0
        atoms = read(f'data/{ff}.POSCAR', format='vasp')
        id_ = fdb.write(atoms, status='submit', runner='slurm:PPM', label=ff)
        runner_data.to_db('database.db', id_)

In [29]:
submit(1, fdb, 'slurm:PPM')

# test

In [44]:
def main(filerec, index, voxl_size, axis, x_min, y_min,
         x_max, y_max, units, filename=None, _rotate=False):
    """
    Parameters:
    filerec: LabCore file record, containing 3D data
    index: index at which slice is made
    voxl_size: size of voxel in units
    axis: axis along which slice is made
    x_min: lower range of index in x direction
    y_min: lower range of index in y direction
    x_max: upper range of index in x direction
    y_max: upper range of index in y direction
    units: the units of voxl size
    filename: the png filename to store
    _rotate: rotate the data 180 to match exp directions
    """
    from matplotlib import pyplot as plt
    import numpy as np
    
    # data sanitisation
    index = int(index)
    voxl_size = list(map(float, voxl_size))
    if len(voxl_size) == 1:
        voxl_size *= 3
    axis = int(axis)
    x_min = int(x_min)
    y_min = int(y_min)
    
    if x_max == 'None':
        x_max = None
    elif x_max is not None:
        x_max = int(x_max)
    
    if y_max == 'None':
        y_max = None
    elif y_max is not None:
        y_max = int(y_max)
    
    # data = filerec['data']
    data = np.load(filerec)
    if _rotate:
        data = np.rot90(data, 2)
    if filename is None:
        filename = '.'.join(filerec.split('.')[:-1])
    a = tuple([slice(None) if i != axis else index for i in range(3)])
    img = data[a]
    a = tuple([slice(x_min, x_max), slice(y_min, y_max)])
    img = img[a]

    plt.imshow(img.T,
           #interpolation=None,
           origin='lower',
           cmap=plt.cm.Greys_r
          )
    ind = [1, 0, 0][axis]
    x = np.arange(0, img.shape[0] * voxl_size[ind], voxl_size[ind])
    nx = x.shape[0]
    n_labels = 5
    step_x = int(nx / (n_labels - 1))
    x_positions = np.arange(0 , nx, step_x)
    x_labels = x[::step_x]
    plt.xticks(x_positions, [f"{i:.2f}" for i in x_labels])
    plt.xlabel(f"{['Y', 'X', 'X'][axis]} {units}")
    
    ind = [2, 2, 1][axis]
    x = np.arange(0, img.shape[1] * voxl_size[ind], voxl_size[ind])
    nx = x.shape[0]
    n_labels = 5
    step_x = int(nx / (n_labels - 1))
    x_positions = np.arange(0 , nx, step_x)
    x_labels = x[::step_x]
    plt.yticks(x_positions, [f"{i:.2f}" for i in x_labels])
    plt.ylabel(f"{['Z', 'Z', 'Y'][axis]} {units}")
    
    plt.title(f"{['X', 'Y', 'Z'][axis]} at {index * voxl_size[axis]:.2f}{units}")
    if filename != 'dummy.png':
        plt.savefig(filename)

In [60]:
@interact(i=(1, 30), z=(40, 65))
def interactive_xy(i, z):
    row = fdb.get(i)
    np.save('test.npy', row.data['box'])
    print(row.label, row.data['runner']['tasks'][0][2]['Amp'])
    main('test.npy', z, [0.1], 2, 0, 0, None, None, 'A',
     'dummy.png', _rotate=True)

interactive(children=(IntSlider(value=15, description='i', max=30, min=1), IntSlider(value=52, description='z'…

In [61]:
row = fdb.get(15)

In [63]:
runnerdata_test = RunnerData.from_db(fdb, 15)

In [67]:
runnerdata_test.add_file('gen_params.py')

In [70]:
for i in range(1, len(fdb)+1):
    runnerdata_test = RunnerData.from_db(fdb, i)
    if runnerdata_test.data['tasks'][0][2]['Amp'] < 5:
        print(i)
        runnerdata_test.data['tasks'][0][2]['gridz'] = 25.0
        runnerdata_test.add_file('gen_params.py')
        runnerdata_test.to_db(fdb, i)
        submit(i, fdb, 'slurm:PPM')

10
11
12
13
14
15
18
19
20
21
22
23
30
